# ESPN Sports API Explorer

This notebook lets you explore sports data from ESPN's public API.

**Supported Sports:** NFL, NBA, MLB, NHL, Soccer (90+ leagues), College Football, College Basketball, UFC, Golf, Racing, Tennis

---

## Setup

First, install the package (run this cell once):

In [ ]:
!pip install espn-sports-api -q

In [ ]:
# Import everything we need
from espn_sports_api import (
    NFL,
    NBA,
    MLB,
    NHL,
    Soccer,
    NCAAF,
    NCAAB,
    quick_scores,
    Odds,
    Conferences,
    parse_injuries,
    parse_teams,
)
from datetime import date, timedelta
import json


# Helper to pretty print JSON
def show(data, max_lines=30):
    """Pretty print JSON data (truncated for readability)"""
    output = json.dumps(data, indent=2)
    lines = output.split("\n")
    if len(lines) > max_lines:
        print("\n".join(lines[:max_lines]))
        print(f"\n... ({len(lines) - max_lines} more lines)")
    else:
        print(output)


print("Ready to explore!")

---

## Quick Start: Today's Scores

Get scores for any sport with one line:

In [ ]:
# Change this to: 'nfl', 'nba', 'mlb', 'nhl', 'ncaaf', 'ncaab', 'mls', 'epl'
SPORT = "nba"

scores = quick_scores(SPORT)

print(f"Today's {SPORT.upper()} Games:\n")
for event in scores.get("events", []):
    name = event["name"]
    status = event.get("status", {}).get("type", {}).get("shortDetail", "")
    print(f"  {name} - {status}")

---

## NFL Explorer

In [ ]:
nfl = NFL()

### Today's Games

In [ ]:
games = nfl.today()

for event in games.get("events", []):
    comp = event.get("competitions", [{}])[0]
    competitors = comp.get("competitors", [])

    if len(competitors) == 2:
        away = next((c for c in competitors if c["homeAway"] == "away"), {})
        home = next((c for c in competitors if c["homeAway"] == "home"), {})

        print(
            f"{away['team']['abbreviation']} {away.get('score', '-')} @ "
            f"{home['team']['abbreviation']} {home.get('score', '-')}"
        )

### All NFL Teams

In [ ]:
teams_data = nfl.teams()
teams = parse_teams(teams_data)

print(f"{len(teams)} NFL Teams:\n")
for team in sorted(teams, key=lambda t: t.name):
    print(f"  {team}")

### Team Details

Change the team abbreviation to explore different teams:

In [ ]:
TEAM = "kc"  # Try: 'buf', 'sf', 'dal', 'phi', 'mia', etc.

team_info = nfl.team(TEAM)
team = team_info.get("team", {})

print(f"Team: {team.get('displayName')}")
print(f"Location: {team.get('location')}")
print(f"Record: {team.get('record', {}).get('items', [{}])[0].get('summary', 'N/A')}")
print(f"Standing: {team.get('standingSummary', 'N/A')}")

### Team Roster

In [ ]:
roster = nfl.team_roster(TEAM)

print(f"{team.get('displayName', TEAM.upper())} Roster:\n")

for group in roster.get("athletes", []):
    position = group.get("position", "Unknown")
    players = group.get("items", [])

    if players:
        print(f"\n{position}:")
        for player in players[:5]:  # Show first 5 per position
            name = player.get("displayName", "Unknown")
            number = player.get("jersey", "-")
            print(f"  #{number} {name}")

### Injuries

In [ ]:
injury_data = nfl.injuries()
injuries = parse_injuries(injury_data)

print(f"NFL Injuries ({len(injuries)} total):\n")
for injury in injuries[:15]:
    print(f"  {injury}")

### Betting Odds

In [ ]:
scoreboard = nfl.scoreboard()
all_odds = Odds.from_scoreboard(scoreboard)

if not all_odds:
    print("No odds available (games may have started)")
else:
    for game in all_odds:
        print(f"\n{game.away_team} @ {game.home_team}")
        if game.spread:
            print(f"  Spread: {game.spread.favorite} {game.spread.spread}")
        if game.moneyline:
            print(f"  ML: Home {game.moneyline.home_odds} / Away {game.moneyline.away_odds}")
        if game.total:
            print(f"  O/U: {game.total.over_under}")

---

## NBA Explorer

In [ ]:
nba = NBA()

### Today's Games

In [ ]:
games = nba.today()

print(f"NBA Games Today ({len(games.get('events', []))}):\n")
for event in games.get("events", []):
    comp = event.get("competitions", [{}])[0]
    competitors = comp.get("competitors", [])
    status = event.get("status", {}).get("type", {}).get("shortDetail", "")

    if len(competitors) == 2:
        away = next((c for c in competitors if c["homeAway"] == "away"), {})
        home = next((c for c in competitors if c["homeAway"] == "home"), {})

        print(
            f"{away['team']['abbreviation']} {away.get('score', '-')} @ "
            f"{home['team']['abbreviation']} {home.get('score', '-')} | {status}"
        )

### Standings

In [ ]:
standings = nba.standings()

for conf in standings.get("children", []):
    print(f"\n{conf.get('name', 'Conference')}:")
    print(f"{'Team':<30} {'W':>3} {'L':>3} {'PCT':>6}")
    print("-" * 45)

    for entry in conf.get("standings", {}).get("entries", [])[:8]:
        team_name = entry.get("team", {}).get("displayName", "Unknown")
        stats = {s["name"]: s.get("value", 0) for s in entry.get("stats", []) if "name" in s}

        wins = int(stats.get("wins", 0))
        losses = int(stats.get("losses", 0))
        pct = stats.get("winPercent", 0)

        print(f"{team_name:<30} {wins:>3} {losses:>3} {pct:>6.3f}")

---

## Premier League Explorer

In [ ]:
epl = Soccer(league="epl")

### League Table

In [ ]:
table = epl.table()

standings = table.get("children", [{}])[0].get("standings", {})
entries = standings.get("entries", [])

print(f"{'#':<3} {'Team':<25} {'P':>3} {'W':>3} {'D':>3} {'L':>3} {'GD':>4} {'Pts':>4}")
print("-" * 55)

for entry in entries:
    team = entry.get("team", {}).get("displayName", "Unknown")
    stats = {s["name"]: s.get("value", 0) for s in entry.get("stats", []) if "name" in s}

    pos = int(stats.get("rank", 0))
    played = int(stats.get("gamesPlayed", 0))
    wins = int(stats.get("wins", 0))
    draws = int(stats.get("ties", 0))
    losses = int(stats.get("losses", 0))
    gd = int(stats.get("pointDifferential", 0))
    points = int(stats.get("points", 0))

    print(f"{pos:<3} {team:<25} {played:>3} {wins:>3} {draws:>3} {losses:>3} {gd:>+4} {points:>4}")

### This Week's Fixtures

In [ ]:
this_week = epl.date_range(date.today(), date.today() + timedelta(days=7))

print("Upcoming EPL Matches:\n")
for event in this_week.get("events", []):
    name = event.get("name", "Unknown")
    match_date = event.get("date", "")[:10]
    venue = event.get("competitions", [{}])[0].get("venue", {}).get("fullName", "")

    print(f"  {match_date}: {name}")
    if venue:
        print(f"              @ {venue}")

### All Premier League Teams

In [ ]:
teams_data = epl.teams()

all_teams = []
for sport in teams_data.get("sports", []):
    for league in sport.get("leagues", []):
        for team_entry in league.get("teams", []):
            team = team_entry.get("team", team_entry)
            all_teams.append(
                {
                    "id": team.get("id"),
                    "name": team.get("displayName"),
                    "abbr": team.get("abbreviation"),
                }
            )

print(f"Premier League Teams ({len(all_teams)}):\n")
for team in sorted(all_teams, key=lambda t: t["name"]):
    print(f"  {team['abbr']:<5} {team['name']} (ID: {team['id']})")

### Other Soccer Leagues

90+ leagues are supported:

In [ ]:
leagues = Soccer.available_leagues()

print(f"{len(leagues)} leagues available:\n")

# Show some popular ones
popular = [
    "epl",
    "la_liga",
    "bundesliga",
    "serie_a",
    "ligue_1",
    "mls",
    "champions_league",
    "europa_league",
    "liga_mx",
    "eredivisie",
]

for key in popular:
    if key in leagues:
        print(f"  {key}: {leagues[key]}")

print(f"\n  ... and {len(leagues) - len(popular)} more")

In [ ]:
# Try another league - change this!
LEAGUE = "la_liga"

league = Soccer(league=LEAGUE)
games = league.today()

print(f"{LEAGUE.upper()} games today: {len(games.get('events', []))}")
for event in games.get("events", []):
    print(f"  - {event['name']}")

---

## College Sports

Filter by conference using names or enums:

In [ ]:
ncaaf = NCAAF()
ncaab = NCAAB()

# See all available conferences
print("NCAAF Conferences:")
for name, conf_id in Conferences.list_all("ncaaf").items():
    print(f"  {name}: {conf_id}")

In [ ]:
# Get SEC football games
CONFERENCE = "SEC"  # Try: 'BIG_TEN', 'ACC', 'BIG_12', 'PAC_12'

sec_games = ncaaf.scoreboard(conference=CONFERENCE)

print(f"{CONFERENCE} Football Games:\n")
for event in sec_games.get("events", [])[:10]:
    print(f"  - {event['name']}")

In [ ]:
# College basketball rankings
rankings = ncaab.rankings()

for poll in rankings.get("rankings", [])[:1]:  # First poll
    print(f"{poll.get('name', 'Rankings')}:\n")

    for rank in poll.get("ranks", [])[:25]:
        team = rank.get("team", {}).get("nickname", "Unknown")
        record = rank.get("recordSummary", "")
        current = rank.get("current", 0)

        print(f"  {current:>2}. {team} ({record})")

---

## Raw Data Explorer

Use this cell to explore any endpoint and see the raw JSON structure:

In [ ]:
# Explore raw API responses
# Uncomment any line below to see the data structure:

# show(nfl.scoreboard())
# show(nfl.team('kc'))
# show(nfl.standings())
# show(nba.news(limit=5))
# show(epl.table())

# Or explore your own:
show(nfl.news(limit=3))

---

## Historical Data

Look up games from specific dates:

In [ ]:
# Games on a specific date
DATE = date(2024, 12, 25)  # Christmas Day

nba_christmas = nba.on_date(DATE)

print(f"NBA Christmas Day {DATE.year}:\n")
for event in nba_christmas.get("events", []):
    name = event.get("name", "Unknown")
    comp = event.get("competitions", [{}])[0]
    competitors = comp.get("competitors", [])

    if len(competitors) == 2:
        away = next((c for c in competitors if c["homeAway"] == "away"), {})
        home = next((c for c in competitors if c["homeAway"] == "home"), {})

        print(
            f"  {away['team']['abbreviation']} {away.get('score', '-')} @ "
            f"{home['team']['abbreviation']} {home.get('score', '-')}"
        )

In [ ]:
# NFL by week
WEEK = 1

week_games = nfl.for_week(WEEK)

print(f"NFL Week {WEEK}:\n")
for event in week_games.get("events", [])[:10]:
    print(f"  - {event['name']}")

---

## Your Turn!

Use the cells below to explore on your own:

In [ ]:
# Your code here!
# Examples to try:
#   NHL().today()
#   MLB().team('nyy')
#   Soccer(league='bundesliga').table()
#   Racing(series='f1').standings()

